In [ ]:
import os
print (os.path.dirname(os.getcwd()))
import urllib.request

In [ ]:
from PIL import Image
from imageai.Prediction.Custom import CustomImagePrediction
from flask import jsonify

In [ ]:
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
import os
from apiclient.http import MediaFileUpload
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import requests
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials_from_google_app.json', ['https://www.googleapis.com/auth/drive']) 
drive_service = build('drive','v3',credentials=creds)

In [ ]:
import numpy as np
import os
from flask import Flask, render_template, request
app = Flask(__name__)
UPLOAD_FOLDER = os.path.basename('uploads')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
@app.route('/')
def home():
    return render_template('home.html')
@app.route('/about',methods=['GET'])
def about():
    if 'id' in request.args:
        id = request.args['id']
        print (id)
    url = id
    mp4file = urllib.request.urlopen(url)

    with open("test.mp4", "wb") as handle:
        handle.write(mp4file.read())
    return render_template('about.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    #os.remove("test.jpg")
    file = request.files['image']
    f = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(f)
    os.rename(f,"test.jpg")
    
    

    return render_template('success.html')
@app.route('/classify', methods=['POST'])
def classify_image():
    if request.method == 'POST':    
            prediction = CustomImagePrediction()
            prediction.setModelTypeAsResNet()
            prediction.setJsonPath("model_class.json")
            prediction.setModelPath("trained_model.h5")

            prediction.loadModel(num_objects=10)


            predictions, probabilities = prediction.predictImage('test.jpg', result_count=3)

            results = {}
            for eachPrediction, eachProbability in zip(predictions, probabilities):
                print(eachPrediction , " : " , eachProbability)
                results[eachPrediction] = float(eachProbability)
            profession = max(results,key=lambda k: results[k])
            print (profession)
            return render_template('result.html', profession=profession)
    
@app.route('/train', methods=['POST'])
def train_model():
    if request.method == 'POST':
        return render_template('train.html')
@app.route('/update', methods=['POST'])
def upload_images():
    if request.method == 'POST':
        text = request.form['profession']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credentials_from_google_app.json', ['https://www.googleapis.com/auth/drive']) 
        drive_service = build('drive','v3',credentials=creds)
        name = text
        typed = 'application/vnd.google-apps.folder'
        msg = "mimeType='" +typed+ "' and " + "name='" +name+"'"
        print (msg)
        #list_file=drive_service.files().list(q="mimeType='application/vnd.google-apps.folder' and name = name",spaces='drive',fields='files(id, name)').execute()

        #get the folder id of the folder whose name matched with the profession
        list_file = drive_service.files().list(q=msg,spaces='drive',fields='files(id, name)').execute()
        print (list_file)
        file_details = list_file['files']
        print (file_details)
        folder_id = file_details[0]['id']
        print (file_details[0]['id'])
        #pace the uploaded image into the folder matching the profession
        file_name = str(np.random.randint(0,100000))
        file_metadata = {'name':text + file_name +'.jpg','parents':[folder_id]}
        media = MediaFileUpload('test.jpg',mimetype='image/jpeg',resumable=True)
        file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print ("file uploaded successfully")
        return render_template('file_upload.html')
if __name__ == '__main__':
    app.run()